# Llama 3.2 3B Instruct - Hugging Face API

Este notebook demonstra como usar o modelo **meta-llama/Llama-3.2-3B-Instruct** via Hugging Face Inference API.

## 1. Instalar dependências

In [1]:
!pip install -q transformers huggingface_hub accelerate

## 2. Configurar autenticação

Você precisa de um token do Hugging Face. Obtenha em: https://huggingface.co/settings/tokens

In [ ]:
import os
from huggingface_hub import login

# Opção 1: Definir como variável de ambiente
#os.environ["HF_TOKEN"] =

# Opção 2: Login interativo (vai pedir o token)
# login()

# Opção 3: Passar o token diretamente (não recomendado para código público)
HF_TOKEN = os.environ.get("HF_TOKEN", None)

if HF_TOKEN:
    login(token=HF_TOKEN)
    print("✅ Autenticado no Hugging Face!")
else:
    print("⚠️ Defina HF_TOKEN ou rode login() para se autenticar")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


✅ Autenticado no Hugging Face!


## 3. Usar via Inference API (modo mais simples)

In [6]:
from huggingface_hub import InferenceClient

client = InferenceClient(token=HF_TOKEN)

# Chat básico
response = client.chat.completions.create(
    model="meta-llama/Llama-3.2-3B-Instruct",
    messages=[
        {"role": "user", "content": "Explique o que é Deep Learning em 3 frases."}
    ]
)

print(response.choices[0].message.content)

Deep Learning é uma subárea da Inteligência Artificial que utiliza redes neurais artificiais para processar e aprender padrões em grandes conjuntos de dados. Essa abordagem permite que os sistemas aprendam a reconhecer padrões e fazer previsões ou classificações com alta precisão, sem a necessidade de programação explícita. A Deep Learning é amplamente utilizada em diversas áreas, como visão computacional, processamento de linguagem natural e reconhecimento de voz.


## 4. Chat com contexto (múltiplas mensagens)

In [7]:
messages = [
    {"role": "system", "content": "Você é um assistente especializado em IA. Seja conciso e técnico."},
    {"role": "user", "content": "Qual a diferença entre RNN e Transformer?"},
]

response = client.chat.completions.create(
    model="meta-llama/Llama-3.2-3B-Instruct",
    messages=messages
)

print(response.choices[0].message.content)

RNN (Redes Neurais Recorrentes) e Transformer são dois tipos de redes neurais usadas em processamento de linguagem natural (NLP) e processamento de texto. Aqui estão as principais diferenças entre eles:

**RNN (Redes Neurais Recorrentes)**

*   São redes neurais que processam sequências de dados de forma recorrente, ou seja, elas consideram a sequência de dados como um todo ao fazer previsões.
*   Usam um parâmetro chamado "estado oculto" para armazenar informações sobre a sequência de dados.
*   São mais difíceis de treinar do que as redes neurais convencionais, pois precisam lidar com a dependência temporal.
*   São mais adequadas para tarefas que envolvem sequências de dados, como previsão de sequências, reconhecimento de padrões e processamento de linguagem natural.

**Transformer**

*   São redes neurais que processam sequências de dados de forma paralela, ou seja, elas processam cada elemento da sequência de forma independente.
*   Não usam um estado oculto, em vez disso, usam at

## 5. Streaming (resposta em tempo real)

In [1]:
print("🤖 Llama 3.2: ", end="")
for chunk in client.chat.completions.create(
    model="meta-llama/Llama-3.2-3B-Instruct",
    messages=[{"role": "user", "content": "Liste 3 aplicações práticas de LLMs."}],
    stream=True
):
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)
print()

🤖 Llama 3.2: 

NameError: name 'client' is not defined

## 6. Com parâmetros avançados

In [9]:
response = client.chat.completions.create(
    model="meta-llama/Llama-3.2-3B-Instruct",
    messages=[{"role": "user", "content": "Gerar um título criativo para um artigo sobre IA."}],
    temperature=0.9,
    max_tokens=50,
    top_p=0.9
)

print(response.choices[0].message.content)

Aqui estão algumas sugestões de títulos criativos para um artigo sobre IA:

1. "A Inteligência Artificial: O Futuro que Estamos Construindo"
2. "O Tempo de IA: Como a Intel


## 7. Usar via API REST direta

In [10]:
import requests

API_URL = "https://api-inference.huggingface.co/models/meta-llama/Llama-3.2-3B-Instruct/v1/chat/completions"

headers = {"Authorization": f"Bearer {HF_TOKEN}"}
payload = {
    "model": "meta-llama/Llama-3.2-3B-Instruct",
    "messages": [{"role": "user", "content": "Diga olá em português."}],
}

response = requests.post(API_URL, headers=headers, json=payload)
result = response.json()
print(result["choices"][0]["message"]["content"])

KeyError: 'choices'

## 8. Carregar modelo localmente (requer GPU)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Carregar tokenizer e modelo
model_name = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=HF_TOKEN,
    torch_dtype=torch.float16,
    device_map="auto"
)

print(f"✅ Modelo carregado: {model_name}")

## 9. Geração local com transformers

In [ ]:
# Formatar prompt no estilo Llama
def format_prompt(user_input, system_prompt=""):
    if system_prompt:
        return f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n{user_input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
    return f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n{user_input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

prompt = format_prompt("Explique transformers em uma frase.", "Seja técnico e conciso.")

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.7,
    do_sample=True,
    top_p=0.9
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

## 10. Pipeline simplificado

In [ ]:
from transformers import pipeline

# Criar pipeline de text generation
pipe = pipeline(
    "text-generation",
    model="meta-llama/Llama-3.2-3B-Instruct",
    token=HF_TOKEN,
    torch_dtype=torch.float16,
    device_map="auto"
)

messages = [{"role": "user", "content": "Qual a capital do Brasil?"}]
outputs = pipe(messages, max_new_tokens=50)
print(outputs[0]["generated_text"][-1]["content"])

## Notas

- **Inference API**: Mais simples, não requer GPU, mas tem limites de rate
- **Modelo local**: Mais controle, sem limites, mas requer GPU (pelo menos 8GB VRAM para 3B)
- **Token**: Necessário aceitar os termos do modelo em https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct